In [3]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 3s (133 kB/s)
Reading package lists... Done
openjdk version "1.8.0_362"
OpenJDK Runtime Environment (build 1.8.0_362-8u362-ga-0ubuntu1~20.04.1-b09)
OpenJDK 64-Bit Server VM (build 25.3

In [45]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MovielensDataSet").getOrCreate()

moviesDF = spark.read.option("header","true").csv("movies.csv")
linksDF = spark.read.option("header","true").csv("links.csv")
ratingsDF = spark.read.option("header","true").csv("ratings.csv")
tagsDF = spark.read.option("header","true").csv("tags.csv")

tagsDF.show()


+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
|    18|    431|        Al Pacino|1462138765|
|    18|    431|         gangster|1462138749|
|    18|    431|            mafia|1462138755|
|    18|   1221|        Al Pacino|1461699306|
|    18|   1221|            Mafia|1461699303|
|    18|   5995|        holocaust|1455735472|
|    18|   5995|       true story|1455735479|
|    18|  44665|     twist ending|1456948283|
|    18|  52604|  Anthony Hopkins|

In [46]:
moviesDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [47]:
joinedDF = tagsDF.join(moviesDF,"movieId","left")

joinedDF = joinedDF.join(ratingsDF, "movieId")

joinedDF.show()

+-------+------+-----+----------+--------------------+------+------+------+----------+
|movieId|userId|  tag| timestamp|               title|genres|userId|rating| timestamp|
+-------+------+-----+----------+--------------------+------+------+------+----------+
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   599|   2.5|1498515769|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   583|   3.0|1481474191|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   564|   5.0|1478451501|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   560|   3.5|1469653998|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   484|   4.5|1342368380|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   477|   4.0|1234150559|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   448|   3.0|1262290001|
|  60756|     2|funny|1445714994|Step Brothers (2008)|Comedy|   424|   4.0|1457846119|
|  60756|     2|funny|1445714994|Step Broth

In [49]:
from pyspark.sql.functions import avg,round

joinedDF = joinedDF.groupBy("tag").agg(round(avg("rating"), 1).alias("average_rating"))

joinedDF.show()

+--------------------+--------------+
|                 tag|average_rating|
+--------------------+--------------+
|               anime|           4.0|
|                hope|           3.4|
|wrongful imprison...|           4.4|
|               1970s|           3.8|
|              freaks|           3.8|
|                 art|           3.7|
|          creativity|           5.0|
|  intelligent sci-fi|           3.8|
|        Heartwarming|           4.1|
|             lyrical|           3.6|
|              ransom|           3.9|
|               mafia|           3.6|
|              sequel|           3.7|
|          Emma Stone|           3.9|
|               scifi|           3.0|
|          television|           3.6|
|           Wolverine|           3.7|
|               macho|           3.6|
|        conversation|           4.2|
|intertwining stor...|           4.2|
+--------------------+--------------+
only showing top 20 rows



In [50]:
avgDF = joinedDF.orderBy("average_rating", ascending=False)

avgDF.show()

+--------------------+--------------+
|                 tag|average_rating|
+--------------------+--------------+
|          creativity|           5.0|
|        human rights|           5.0|
|          procedural|           5.0|
|    free to download|           5.0|
|         no dialogue|           5.0|
|   thought provoking|           4.8|
|            jon hamm|           4.8|
|             parrots|           4.8|
|            Dystopia|           4.8|
| movies about movies|           4.7|
|interracial marriage|           4.5|
|   No DVD at Netflix|           4.5|
|              subway|           4.5|
|        good writing|           4.5|
|               italy|           4.5|
|               crazy|           4.5|
|            creative|           4.5|
|   building a family|           4.5|
|reciprocal spectator|           4.5|
|political right v...|           4.5|
+--------------------+--------------+
only showing top 20 rows

